>Trova popolazione in base al nome del comune

In [ ]:
import json
import requests
import re

with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Funzione per trovare l'id in base al name
def find_id_comune_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^\d{6}$", code['id']):
            return code['id']
    return None

# Trova id tramite nome comune
comune_to_find = 'Salerno'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")


# Funzione per ottenere l'observation dal JSON della risposta
def extract_abitanti_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/22_289/.TOTAL.{id_comune_found}.9.99.?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_abitanti_observation(response.json())

    if observation is not None:
        print(f"L'observation della popolazione estratto è: {observation}")
    else:
        print("Observation della popolazione non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")




L'id corrispondente al nome 'Salerno' è: 065116
L'observation estratto è: 127485


Trova presenza di problemi nella zona di residenza delle famiglie, come inquinamento, rumori o criminalità.<br>
Trova numero segnalazioni di denunce e arresti.

In [ ]:
import json
import requests
import re
from enum import Enum

with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Funzione per trovare l'id in base al name
def find_id_regione_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^IT.*$", code['id']):
            return code['id']
    return None

# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = 'Campania'
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")


# Funzione per ottenere l'observation dal JSON della risposta
def extract_problemi_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

class Problema(Enum):
    INQUINAMENTO = 1
    RUMORI = 2
    CRIMINALITA = 3

problema_scelto= Problema.INQUINAMENTO

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/33_291/A.{id_regione_found}...{problema_scelto.value}..........?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_problemi_observation(response.json())

    if observation is not None:
        print(f"L'observation di {problema_scelto.name} estratto è: {observation}")
    else:
        print(f"Observation di {problema_scelto.name} non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


#### Segnalazioni relative a persone denunciate e arresti

def extract_denunce_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/73_342/A.{id_regione_found}.REPN.TOT.?startPeriod=2022&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_denunce_observation(response.json())

    if observation is not None:
        print(f"L'observation delle denunce estratto è: {observation}")
    else:
        print(f"Observation delle denunce non trovato nella risposta JSON.")
else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Campania' è: ITF3
L'observation di INQUINAMENTO estratto è: 22.3
L'observation delle denunce estratto è: 71256


Trova spesa media per consumi

In [ ]:
import json
import requests
import re
from enum import Enum

with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Funzione per trovare l'id in base al name
def find_id_regione_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^IT.*$", code['id']):
            return code['id']
    return None

# Trova id tramite nome regione oppure Nord-Centro-Sud
regione_to_find = 'Campania'
id_regione_found = find_id_regione_by_name(regione_to_find)

if id_regione_found:
    print(f"L'id corrispondente al nome '{regione_to_find}' è: {id_regione_found}")
else:
    print(f"Nome '{regione_to_find}' non trovato.")


# Funzione per ottenere l'observation dal JSON della risposta
def extract_spesamedia_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0:0:0:0:0:0:0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/31_739/A.HH.99.SPESA_MEDIA..ALL..{id_regione_found}.ALL.ALL.TOT?startPeriod=2021&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_spesamedia_observation(response.json())

    if observation is not None:
        print(f"L'observation della spesa media estratto è: {observation}")

        costo_vita= None
        if observation < 2100:
          costo_vita= "BASSO"
        elif observation > 2700:
          costo_vita= "ALTO"
        else:
          costo_vita= "MEDIO"
        print(f"Il Costo della Vita corrisponde a: {costo_vita}")
    else:
        print(f"Observation della spesa media non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Campania' è: ITF3
L'observation della spesa media estratto è: 2034.65
Il Costo della Vita corrisponde a: BASSO


Trova aree a pericolosità idraulica in km2

In [ ]:
import json
import requests
import re
from enum import Enum

with open('CL_ITTER107.json', 'r') as file:
    data = json.load(file)

# Estrai la lista di codici dal file JSON
codes = data['data']['codelists'][0]['codes']

# Funzione per trovare l'id in base al name
def find_id_comune_by_name(name):
    for code in codes:
        if code['name']['it'] == name and re.match("^\d{6}$", code['id']):
            return code['id']
    return None

# Trova id tramite nome comune
comune_to_find = 'Venezia'
id_comune_found = find_id_comune_by_name(comune_to_find)

if id_comune_found:
    print(f"L'id corrispondente al nome '{comune_to_find}' è: {id_comune_found}")
else:
    print(f"Nome '{comune_to_find}' non trovato.")

# Funzione per ottenere l'observation dal JSON della risposta
def extract_superficie_observation(response_json):
    try:
        observation = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        return observation
    except KeyError:
        return None

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_1050/A.{id_comune_found}.TOTAREA?startPeriod=2023&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

superficie= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observation = extract_superficie_observation(response.json())

    if observation is not None:
        print(f"L'observation della superficie estratto è: {observation}")
        superficie= observation/100
    else:
        print(f"Observation della superficie non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


# Funzione per ottenere l'observation dal JSON della risposta
def extract_rischi_idro_observation(response_json):
    try:
        observationHIGH = response_json['data']['dataSets'][0]['series']['0:0:0']['observations']['0'][0]
        observationLOW = response_json['data']['dataSets'][0]['series']['0:0:1']['observations']['0'][0]
        observationMED = response_json['data']['dataSets'][0]['series']['0:0:2']['observations']['0'][0]
        observations= [observationLOW, observationMED, observationHIGH]
        return observations
    except KeyError:
        return None

# Costruisci l'URL con l'id trovato
api_url = f'https://sdmx.istat.it/SDMXWS/rest/data/729_995/A.{id_comune_found}.LOWFLHZ+MEDFLHZ+HIGHFLHZ?startPeriod=2017&format=jsondata'

# Effettua la richiesta all'API
response = requests.get(api_url)

rischio_idro= None
# Verifica se la richiesta ha avuto successo (status code 200)
if response.status_code == 200:
    # Estrai l'observation dalla risposta JSON
    observations = extract_rischi_idro_observation(response.json())

    if observations is not None:
        print(f"L'observationLOW estratto è: {observations[0]}")
        print(f"L'observationMED estratto è: {observations[1]}")
        print(f"L'observationHIGH estratto è: {observations[2]}")

        rischio_idro= (observations[0] + observations[1] + observations[2])/superficie*100
        print(f"Il rischio idrogeologico è: {rischio_idro}")

    else:
        print(f"Observation non trovato nella risposta JSON.")

else:
    print(f"Errore nella richiesta all'API. Codice di stato: {response.status_code}")


L'id corrispondente al nome 'Venezia' è: 027042
L'observation della superficie estratto è: 41589.27
L'observationLOW estratto è: 128.4
L'observationMED estratto è: 24.97
L'observationHIGH estratto è: 12.11
Il rischio idrogeologico è: 39.78910906587204


Estrazione Zone Sismiche.

In [85]:
import pandas as pd

# Caricamento dei dati
sismi = pd.read_csv('italiaMedianaSismi.csv', sep=';')

def convert_percent(x):
    return float(x.replace(",", "."))

sismi["ag"] = sismi["ag"].apply(convert_percent)
sismi["LON"] = sismi["LON"].apply(convert_percent)
sismi["LAT"] = sismi["LAT"].apply(convert_percent)

min_value = sismi["ag"].min()
max_value = sismi["ag"].max()
sismi["normalized_ag"] = 4 * (sismi["ag"] - min_value) / (max_value - min_value)

def calcola_rischio(mediana):
    if mediana >= 3.0:
        return 1
    elif 2.0 <= mediana < 3.0:
        return 2
    elif 1.0 <= mediana < 2.0:
        return 3
    else:
        return 4

#Maggiore il numero, più il rischio
sismi['Zona Sismica']= sismi['normalized_ag'].apply(calcola_rischio)

print(sismi[["LON", "LAT", "Zona Sismica"]])
sismi[["LON", "LAT", "Zona Sismica"]].to_csv("Zone Sismiche.csv", index=False)

            LON      LAT  Zona Sismica
0       12.0899  47.0710             4
1       12.1193  47.0713             4
2       12.1487  47.0716             4
3       12.1781  47.0719             4
4       12.2075  47.0723             4
...         ...      ...           ...
104560  12.6943  35.3374             4
104561  12.7187  35.3376             4
104562  12.7432  35.3378             4
104563  12.7676  35.3380             4
104564  12.7679  35.3179             4

[104565 rows x 3 columns]


In [91]:
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.neighbors import BallTree
import numpy as np

def trova_coordinate(comune):
    geolocator = Nominatim(user_agent="FindYourPlace")
    location = geolocator.geocode(comune)

    if location:
        latitudine, longitudine = location.latitude, location.longitude
        return latitudine, longitudine
    else:
        return None

comune_da_cercare = "Avezzano"
coordinate = trova_coordinate(comune_da_cercare)

if coordinate:
    print(f"Le coordinate di {comune_da_cercare} sono: {coordinate}")
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")


def trova_coordinate_vicine(latitudine, longitudine, dataframe):
    bt = BallTree(np.deg2rad(dataframe[['LAT', 'LON']].values), metric='haversine')
    distanze, indici = bt.query(np.deg2rad(np.c_[latitudine, longitudine]))

    nearest = dataframe.iloc[indici[0]]

    return nearest[['LAT', 'LON', 'Zona Sismica']]

coordinate_di_riferimento = trova_coordinate(comune_da_cercare)

if coordinate_di_riferimento:
    zone_sismiche = pd.read_csv('Zone Sismiche.csv')

    # Trova le coordinate più vicine al punto di riferimento
    near_coords = trova_coordinate_vicine(coordinate_di_riferimento[0], coordinate_di_riferimento[1], zone_sismiche)

    print(f"Le coordinate più vicine a {comune_da_cercare} sono:")
    print(near_coords)
else:
    print(f"Coordinate non trovate per {comune_da_cercare}")

Le coordinate di Avezzano sono: (42.0347963, 13.4265048)
Le coordinate più vicine a Avezzano sono:
           LAT      LON  Zona Sismica
46316  42.0338  13.4203             1
